In [17]:
import sys
import os
import logging
sys.path.insert(1, os.path.join(sys.path[0], '../../src'))
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from darts.dataprocessing.transformers.scaler import Scaler
from sklearn.preprocessing import MinMaxScaler
from darts import concatenate
from datasets import PaloAltoDataset, WeatherEcadDataset, ShellDataset
from visualization import plot_time_series_predictions
from eval import print_metrics
### Load Dataset
dataset = ShellDataset(subset=9)
data = dataset.load_dataset(split=0.1)

data['train'] = data['train'][5:]
data['test'] = data['test'][5:]
# Transform splits
target_scaler = Scaler(MinMaxScaler())
train_series = target_scaler.fit_transform(data['train'])
test_series = target_scaler.transform(data['test'])

# Load Covariates
weather_dataset = WeatherEcadDataset()
past_covariates = weather_dataset.load_covariates()

# Scale Covariates
covariates_scaler = Scaler(MinMaxScaler())
past_covariates = covariates_scaler.fit_transform(past_covariates)
print("Number of data points in train, test: ", len(data['train'][0]),  len(data['test'][0]))

Loaded 9 time series each with length 365
Number of data points in train, test:  37 328


In [2]:
from darts.models import NHiTSModel

In [7]:
model = NHiTSModel.load("../../models/nhits_model_shell_4.pt")

In [18]:
predictions = []
for train_series_single, test_series_single in zip(train_series, test_series):
    forecast = model.historical_forecasts(
        series=concatenate([train_series_single, test_series_single]),
        past_covariates=past_covariates,
        start=test_series_single.start_time(),
        forecast_horizon=1,
        retrain=False
    )
    predictions.append(forecast)
predictions = target_scaler.inverse_transform(predictions)

c:\Users\tijmen.vanetten\AppData\Local\miniconda3\envs\darts\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\tijmen.vanetten\AppData\Local\miniconda3\envs\darts\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\tijmen.vanetten\AppData\Local\miniconda3\envs\darts\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: P